In [1]:
!pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.0
    Uninstalling torchvision-0.15.0:
      Successfully uninstalled torchvision-0.15.0


In [2]:
import numpy as np
import torch

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [6]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [7]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.0643,  1.9573,  0.8715],
        [-0.3195, -0.5616,  0.7949]], requires_grad=True)
tensor([-0.5398,  0.0615], requires_grad=True)


In [8]:
#We can define the model as follows:
def model(x):
    return x @ w.t() + b

In [10]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[163.3795, -26.7061],
        [221.6272, -27.5567],
        [306.6894, -56.8817],
        [109.3111, -27.2630],
        [243.9293, -20.2508]], grad_fn=<AddBackward0>)


In [11]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases.

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

1. Calculate the difference between the two matrices (preds and targets).
2. Square all elements of the difference matrix to remove negative values.
3. Calculate the average of the elements in the resulting matrix.

- The result is a single number, known as the mean squared error (MSE).

In [12]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [13]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(17947.2441, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True. We'll see how this is useful in just a moment.

In [14]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [16]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.0643,  1.9573,  0.8715],
        [-0.3195, -0.5616,  0.7949]], requires_grad=True)
tensor([[ 11118.9219,  12400.7168,   7519.8013],
        [-10288.2070, -11873.5645,  -7104.8828]])


<h1>Adjust weights and biases to reduce the loss</h1>
<p>The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's slope w.r.t. the weights and biases.</p><br>
If a gradient element is positive:
<ul>
    <li>increasing the weight element's value slightly will increase the loss</li>
    <li>decreasing the weight element's value slightly will decrease the loss</li>
</ul>
If a gradient element is negative:
<ul>
    <li>increasing the weight element's value slightly will decrease the loss</li>
    <li>easing the weight element's value slightly will increase the loss</li>
 </ul>
 
 <p>The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the gradient descent optimization algorithm that we'll use to improve our model (by descending along the gradient).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.</p>

In [18]:
w
w.grad

tensor([[ 11118.9219,  12400.7168,   7519.8013],
        [-10288.2070, -11873.5645,  -7104.8828]])

In [19]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [20]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(17947.2441, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [21]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

Let's implement the above step by step.

In [22]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[143.7194,  -8.1841],
        [195.7823,  -3.1973],
        [276.0362, -27.8983],
        [ 89.8538,  -9.0333],
        [219.0874,   3.2213]], grad_fn=<AddBackward0>)


In [23]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(12199.2314, grad_fn=<DivBackward0>)


In [24]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 9088.3945, 10216.5986,  6172.5620],
        [-8374.3340, -9812.4512, -5834.0771]])
tensor([ 108.6958, -101.0183])


In [25]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [26]:
# New weights and biases
print(w)
print(b)

tensor([[-0.2664,  1.7311,  0.7346],
        [-0.1329, -0.3447,  0.9243]], requires_grad=True)
tensor([-0.5422,  0.0638], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(8324.4473, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. each iteration is called an epoch.

In [28]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [29]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(97.9671, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [30]:
# Predictions
preds

tensor([[ 55.2844,  73.6713],
        [ 78.8125, 105.8595],
        [129.4348, 115.7513],
        [ 10.4681,  56.7868],
        [102.1011, 116.6406]], grad_fn=<AddBackward0>)

In [31]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs.